In [1]:
!pip install pymupdf 

In [2]:
! pip install PyPDF2 python-docx

In [ ]:
# =======================
# CV/LinkedIn Matcher + Clean CSV + Gradio GUI (single cell)
# =======================

import os, re, json, tempfile, logging
from typing import List, Dict, Any

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
import PyPDF2, docx

# ---- optional OCR (PNG/JPG/WEBP) ----
OCR_AVAILABLE = False
try:
    from PIL import Image
    import pytesseract
    tpath = os.environ.get("TESSERACT_PATH")
    if tpath:
        pytesseract.pytesseract.tesseract_cmd = tpath  # e.g. "C:/Program Files/Tesseract-OCR/tesseract.exe"
    OCR_AVAILABLE = True
except Exception:
    OCR_AVAILABLE = False

# Reduce library chattiness
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)

# ========= Base system =========
class CVMatchingSystem:
    def __init__(self):
        print("Loading Qwen 2.5 1.5B model...")
        self.model_name = "Qwen/Qwen2.5-1.5B-Instruct"
        dtype = torch.float16 if torch.cuda.is_available() else torch.float32
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=dtype,
            device_map="auto"
        )
        print("Loading Sentence Transformer models...")
        self.responsibilities_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.degree_model = SentenceTransformer('all-MiniLM-L6-v2')
        print("All models loaded successfully!")

    # ---------- OCR ----------
    def extract_text_from_image(self, file_path: str) -> str:
        if not OCR_AVAILABLE:
            return ""
        try:
            img = Image.open(file_path)
            txt = pytesseract.image_to_string(img)
            return txt or ""
        except Exception:
            return ""

    def extract_text_from_file(self, file_path):
        """PDF/DOCX/TXT/RTF/PNG/JPG/JPEG/WEBP"""
        text = ""
        try:
            lower = file_path.lower()
            if lower.endswith('.pdf'):
                with open(file_path, 'rb') as f:
                    reader = PyPDF2.PdfReader(f)
                    for page in reader.pages:
                        text += (page.extract_text() or "") + "\n"
            elif lower.endswith('.docx'):
                doc_ = docx.Document(file_path)
                for p in doc_.paragraphs:
                    text += p.text + "\n"
            elif lower.endswith(('.txt', '.rtf')):
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    text = f.read()
            elif lower.endswith(('.png', '.jpg', '.jpeg', '.webp')):
                text = self.extract_text_from_image(file_path)
            else:
                return None
        except Exception:
            return None
        return text

    @staticmethod
    def _norm_spaces(s: str) -> str:
        return re.sub(r'\s+', ' ', s or '').strip()

    def split_text(self, text):
        text = self._norm_spaces(text)
        if not text:
            return "", ""
        mid = len(text)//2
        pos = mid
        for i in range(mid, min(mid+500, len(text))):
            if text[i] in ".!?" and (i+1 >= len(text) or text[i+1] == " "):
                pos = i+1; break
        if pos == mid:
            for i in range(mid, max(mid-500, 0), -1):
                if text[i] in ".!?" and (i+1 >= len(text) or text[i+1] == " "):
                    pos = i+1; break
        return text[:pos].strip(), text[pos:].strip()

    def get_first_10_lines(self, text):
        lines = re.split(r'\r|\n', text or "")
        if len(lines) == 1:
            chunk = 120
            lines = [text[i:i+chunk] for i in range(0, len(text), chunk)]
        return "\n".join(lines[:10])

    def extract_years_from_pattern(self, text):
        pats = [
            r'(\d+)\s*\+\s*years?\s+of\s+experience',
            r'\+\s*(\d+)\s+years?\s+of\s+experience',
            r'(\d+)\s+years?\s+of\s+experience',
            r'(\d+)\s*\+\s*year\s+of\s+experience',
            r'\+\s*(\d+)\s+year\s+of\s+experience',
            r'(\d+)\s+year\s+of\s+experience'
        ]
        maxy = 0
        for p in pats:
            m = re.findall(p, text or "", re.I)
            if m:
                ys = [int(x) for x in m]
                maxy = max(maxy, max(ys))
        return maxy

    def query_llm(self, prompt):
        try:
            messages = [
                {"role": "system", "content": "You are a helpful assistant that extracts specific information from CVs/resumes. Be concise."},
                {"role": "user", "content": prompt}
            ]
            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
            out = self.model.generate(
                **inputs, max_new_tokens=256, do_sample=True, temperature=0.1,
                top_p=0.9, pad_token_id=self.tokenizer.eos_token_id
            )
            out = [o[len(i):] for i, o in zip(inputs.input_ids, out)]
            return self.tokenizer.batch_decode(out, skip_special_tokens=True)[0].strip()
        except Exception:
            return ""

    def query_llm_batch(self, prompts, max_new_tokens=128):
        try:
            sys = {"role": "system", "content": "You extract info from CVs/LinkedIn. Be concise."}
            texts = []
            for p in prompts:
                msgs = [sys, {"role": "user", "content": p}]
                texts.append(self.tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True))
            inputs = self.tokenizer(texts, return_tensors="pt", padding=True).to(self.model.device)
            out = self.model.generate(
                **inputs, max_new_tokens=max_new_tokens, do_sample=False,
                temperature=0.0, top_p=1.0, pad_token_id=self.tokenizer.eos_token_id
            )
            res = []
            for i in range(len(texts)):
                gen_only = out[i, inputs.input_ids[i].shape[0]:]
                res.append(self.tokenizer.decode(gen_only, skip_special_tokens=True).strip())
            return res
        except Exception:
            return ["" for _ in prompts]

    def create_feature_prompt(self, cv_part, feature_name, context=""):
        base = f"Extract information from this CV text: {context}\n\n{cv_part}\n\n"
        prompts = {
            "Responsibilities": base + "Extract key job responsibilities and work duties. Return concise description:",
            "Degree":          base + "Extract highest educational degree (Bachelor's/Master's/PhD). Format: 'Degree in Major':",
            "Skills":          base + "Extract technical and professional skills as comma-separated values:",
            "GPA":             base + "Extract GPA number only:",
            "Achievements":    base + "Extract achievements and awards as comma-separated values:",
            "Volunteering":    base + "Extract volunteering experiences as comma-separated values:",
            "Candidate Name":  base + "Extract the person's FULL NAME only (no extra words):",
            "Current Title":   base + "Extract the most recent or primary job title (short):",
            "LinkedIn URL":    base + "Extract the LinkedIn profile URL, if present. Return only the URL or empty:",
            "Contact":         base + "Extract email and phone in one line 'email | phone' (leave blank if missing):",
        }
        return prompts.get(feature_name, base + f"{feature_name}:")

    def extract_feature_from_part(self, cv_part, feature_name, context=""):
        return self.query_llm(self.create_feature_prompt(cv_part, feature_name, context))

    def combine_feature_results(self, a, b, feature_name):
        if feature_name in ["Skills", "Achievements", "Volunteering"]:
            combined = []
            for r in [a, b]:
                if r and r.strip():
                    items = [i.strip() for i in r.split(",") if i.strip()]
                    combined.extend(items)
            seen = set()
            out = []
            for i in combined:
                k = i.lower()
                if k in seen: 
                    continue
                seen.add(k); out.append(i)
            return out
        elif feature_name == "GPA":
            g1 = self.extract_gpa(a) if a else 0.0
            g2 = self.extract_gpa(b) if b else 0.0
            return f"{max(g1, g2):.2f}" if max(g1, g2) > 0 else "2.00"
        else:
            a, b = (a or "").strip(), (b or "").strip()
            if not a and not b: return ""
            if not a: return b
            if not b: return a
            return a if len(a) >= len(b) else b

    def extract_gpa(self, text):
        pats = [r'GPA\s*[:]?\s*(\d\.\d+)', r'(\d\.\d+)\s*GPA', r'(\d\.\d+)\s*/\s*4\.0']
        for p in pats:
            m = re.findall(p, text or "", re.I)
            if m:
                return max(float(x) for x in m)
        dec = re.findall(r'\b(\d\.\d+)\b', text or "")
        if dec:
            cand = [float(x) for x in dec if 1.0 <= float(x) <= 4.0]
            return max(cand) if cand else 0.0
        return 0.0

    def parse_contacts_and_links(self, text: str) -> Dict[str, str]:
        linkedin = ""
        m = re.search(r'(https?://(?:www\.)?linkedin\.com/[A-Za-z0-9/_\-\?\=&%\.]+)', text or "", re.I)
        if m: linkedin = m.group(1).strip().rstrip(').,;')
        emails = re.findall(r'[A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,}', text or "")
        phones = re.findall(r'(\+?\d[\d\s\-\(\)]{7,}\d)', text or "")
        email = emails[0] if emails else ""
        phone = re.sub(r'[^\d+]', '', phones[0]) if phones else ""
        contact = " | ".join([p for p in [email, phone] if p])
        return {"LinkedIn URL": linkedin, "Email": email, "Phone": phone, "Contact": contact}

    def extract_job_requirements(self, job_description: str):
        """LLM extraction from JD. If JD empty -> neutral requirements."""
        if not (job_description or "").strip():
            return {
                "Responsibilities": "",
                "Degree": "",
                "Years of Experience": 0,
                "Skills": [],
                "Plus Skills": []
            }
        prompts = {
            "Responsibilities": f"Extract key responsibilities from job description:\n{job_description}\nResponsibilities:",
            "Degree": f"Extract required degree from job description:\n{job_description}\nRequired Degree:",
            "Years of Experience": f"Extract required years of experience from job description:\n{job_description}\nYears Required:",
            "Skills": f"Extract required skills from job description as comma-separated values:\n{job_description}\nRequired Skills:",
            "Plus Skills": f"Extract preferred/plus skills from job description as comma-separated values:\n{job_description}\nPlus Skills:"
        }
        res = {k: self.query_llm(p) for k, p in prompts.items()}
        years = 0
        years_text = res.get("Years of Experience", "")
        for pat in [r'(\d+)\s*\+\s*years?', r'(\d+)\s+years?', r'(\d+)\s*[-]+\s*years?']:
            m = re.findall(pat, years_text or "", re.I)
            if m:
                years = max(int(x) for x in m); break
        res["Years of Experience"] = years
        for s in ["Skills", "Plus Skills"]:
            if res.get(s):
                res[s] = list({i.strip() for i in res[s].split(",") if i.strip()})
            else:
                res[s] = []
        return res

    def calculate_matching_scores(self, candidate_features, job_requirements):
        scores = {}
        # 1. Responsibilities (19%)
        rc = candidate_features.get("Responsibilities", "")
        rj = job_requirements.get("Responsibilities", "")
        if rc and rj:
            e1 = self.responsibilities_model.encode(rc, convert_to_tensor=True)
            e2 = self.responsibilities_model.encode(rj, convert_to_tensor=True)
            scores["responsibilities"] = util.pytorch_cos_sim(e1, e2).item()
        else:
            scores["responsibilities"] = 0.0
        # 2. Degree (19%)
        dc = candidate_features.get("Degree", "")
        dj = job_requirements.get("Degree", "")
        if dc and dj:
            e1 = self.degree_model.encode(dc, convert_to_tensor=True)
            e2 = self.degree_model.encode(dj, convert_to_tensor=True)
            scores["degree"] = util.pytorch_cos_sim(e1, e2).item()
        else:
            scores["degree"] = 0.0
        # 3. Experience (19%)
        ec = candidate_features.get("Years of Experience", 0)
        ej = job_requirements.get("Years of Experience", 0)
        scores["experience"] = 1.0 if ec >= ej else ec / max(ej, 1)
        # 4. Skills (19%)
        sc = set(candidate_features.get("Skills", []))
        sj = set(job_requirements.get("Skills", []))
        scores["skills"] = (len(sc & sj) / len(sj)) if sj else 0.0
        # 5. Plus skills (5%)
        pc = set(candidate_features.get("Plus Skills", []))
        pj = set(job_requirements.get("Plus Skills", []))
        scores["plus_skills"] = (len(pc & pj) / len(pj)) if pj else 0.0
        # 6. Extra skills (5%)
        allc = set(candidate_features.get("Skills", []) + candidate_features.get("Plus Skills", []))
        req = sj | pj
        extra = allc - req
        scores["extra_skills"] = min(len(extra)/10, 1.0)
        # 7. GPA (5%)
        gpa = float(candidate_features.get("GPA", "2.00") or 2.0)
        scores["gpa"] = min(gpa/4.0, 1.0)
        # 8. Achievements (6%)
        ach = len(candidate_features.get("Achievements", []))
        scores["achievements"] = min(ach/5.0, 1.0)
        # 9. Volunteering (3%)
        vol = len(candidate_features.get("Volunteering", []))
        scores["volunteering"] = min(vol/2.0, 1.0)
        return scores

    def calculate_final_score(self, scores):
        weights = {
            'responsibilities': 0.19, 'degree': 0.19, 'experience': 0.19, 'skills': 0.19,
            'plus_skills': 0.05, 'extra_skills': 0.05, 'gpa': 0.05, 'achievements': 0.06, 'volunteering': 0.03
        }
        total = 0.0
        for k, w in weights.items():
            total += scores.get(k, 0.0) * w
        return min(total, 1.0)

    def process_cv(self, file_path):
        txt = self.extract_text_from_file(file_path)
        if not txt:
            return None
        guess = self.parse_contacts_and_links(txt)
        p1, p2 = self.split_text(txt)
        first10 = self.get_first_10_lines(p1)
        years = self.extract_years_from_pattern(first10)
        feats = {"Years of Experience": years if years > 0 else 0}
        # core features
        core = ["Responsibilities", "Degree", "Skills", "GPA", "Achievements", "Volunteering"]
        for f in core:
            r1 = self.extract_feature_from_part(p1, f)
            ctx = f"First part found: {r1[:100]}" if r1 else ""
            r2 = self.extract_feature_from_part(p2, f, ctx)
            feats[f] = self.combine_feature_results(r1, r2, f)
        feats["Plus Skills"] = feats.get("Skills", [])[:]
        # identity + contact
        id_feats = ["Candidate Name", "Current Title", "LinkedIn URL", "Contact"]
        prompts1 = [self.create_feature_prompt(p1, f) for f in id_feats]
        res1 = self.query_llm_batch(prompts1, max_new_tokens=96)
        prompts2 = [
            self.create_feature_prompt(p2, feat, context=f"First part found: {a[:80]}")
            for feat, a in zip(id_feats, res1)
        ]
        res2 = self.query_llm_batch(prompts2, max_new_tokens=96)
        for feat, a, b in zip(id_feats, res1, res2):
            feats[feat] = self.combine_feature_results(a, b, feat)
        # fill from regex if missing
        feats.setdefault("LinkedIn URL", guess.get("LinkedIn URL", ""))
        if not feats.get("LinkedIn URL"): feats["LinkedIn URL"] = guess.get("LinkedIn URL", "")
        if not feats.get("Contact"):       feats["Contact"] = guess.get("Contact", "")
        return feats

    def process_candidates(self, folder_path, job_description):
        if not os.path.isdir(folder_path):
            raise FileNotFoundError(f"Folder not found: {folder_path}")
        job_req = self.extract_job_requirements(job_description)
        exts = ['.pdf', '.docx', '.txt', '.rtf', '.png', '.jpg', '.jpeg', '.webp']
        files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                 if any(f.lower().endswith(x) for x in exts)]
        if not files:
            return []
        results = []
        for f in files:
            feats = self.process_cv(f)
            if not feats: 
                continue
            scores = self.calculate_matching_scores(feats, job_req)
            results.append({
                "filename": os.path.basename(f),
                "features": feats,
                "scores": scores,
                "final_score": self.calculate_final_score(scores)
            })
        results.sort(key=lambda x: x['final_score'], reverse=True)
        return results

# ========= Clean CSV helpers =========
import pandas as pd

def _strip_ws(s: str) -> str:
    s = (s or "")
    s = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]", " ", s)  # drop control chars
    s = s.replace("\r", " ").replace("\n", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _clean_cell(x, replace_commas=True, max_len=180) -> str:
    if isinstance(x, (list, tuple, set)):
        x = " | ".join([_strip_ws(str(i)) for i in x if str(i).strip()])
    else:
        x = _strip_ws(str(x))
    if replace_commas:
        x = x.replace(",", " ·")
    if max_len and len(x) > max_len:
        x = x[: max_len - 1] + "…"
    return x

def _norm_skills(skills, topn=10):
    out, seen = [], set()
    for s in skills or []:
        s = _strip_ws(str(s))
        s = re.sub(r"^[\-\•\·\|]+", "", s).strip(" ,;:|-")
        if not s: 
            continue
        k = s.lower()
        if k in seen: 
            continue
        seen.add(k); out.append(s)
        if len(out) >= topn:
            break
    return out

def _split_contact(contact: str):
    contact = _strip_ws(contact)
    email_match = re.search(r"[A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,}", contact)
    phone_match = re.search(r"(\+?\d[\d\s\-\(\)]{7,}\d)", contact)
    email = email_match.group(0) if email_match else ""
    phone = re.sub(r"[^\d+]", "", phone_match.group(0)) if phone_match else ""
    return email, phone

def build_summary_df(results, topk: int = 10) -> pd.DataFrame:
    top = results[:topk]
    rows = []
    for i, r in enumerate(top, 1):
        feats = r.get("features", {}) or {}
        skills = _norm_skills(feats.get("Skills", []), topn=10)
        email, phone = _split_contact(feats.get("Contact", ""))

        row = {
            "rank": i,
            "filename": _clean_cell(r.get("filename", ""), True, 120),
            "name": _clean_cell(feats.get("Candidate Name", ""), True, 80),
            "title": _clean_cell(feats.get("Current Title", ""), True, 100),
            "linkedin": _clean_cell(feats.get("LinkedIn URL", ""), False, 140),
            "email": _clean_cell(email, True, 120),
            "phone": _clean_cell(phone, True, 30),
            "final_score(%)": round(100 * float(r.get("final_score", 0.0)), 2),
            "years_exp": int(feats.get("Years of Experience", 0) or 0),
            "degree": _clean_cell(feats.get("Degree", ""), True, 120),
            "gpa": _clean_cell(feats.get("GPA", ""), True, 8),
            "skills_top10": _clean_cell(" | ".join(skills), True, 300),
        }
        rows.append(row)

    cols = ["rank","filename","linkedin","email","phone", "final_score(%)"]
    return pd.DataFrame(rows, columns=cols).fillna("")

def save_outputs(summary_df: pd.DataFrame, results):
    out_dir = tempfile.mkdtemp(prefix="cv_match_out_")
    csv_path = os.path.join(out_dir, "cv_matching_summary.csv")
    json_path = os.path.join(out_dir, "cv_matching_results.json")
    summary_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(results[:10], f, ensure_ascii=False, indent=2)
    return csv_path, json_path

# ========= Gradio UI =========
import gradio as gr

# cache the heavy system
_SYSTEM = None
def get_system():
    global _SYSTEM
    if _SYSTEM is None:
        _SYSTEM = CVMatchingSystem()
    return _SYSTEM

DEFAULT_JD = ""  # JD optional; leave blank if you don't want to use one

def run_match(files: list, jd_text: str, topk: int):
    if not files:
        return "Please upload at least one file.", None, None, None

    # write uploads to a temp folder
    work_dir = tempfile.mkdtemp(prefix="cvs_")
    for f in files:
        # f is a tempfile.NamedTemporaryFile or path-like (gradio)
        src = f.name if hasattr(f, "name") else str(f)
        dst = os.path.join(work_dir, os.path.basename(src))
        with open(src, "rb") as r, open(dst, "wb") as w:
            w.write(r.read())

    # run matcher
    sys = get_system()
    results = sys.process_candidates(work_dir, jd_text or "")
    if not results:
        return "No readable candidate files were found.", None, None, None

    df = build_summary_df(results, topk=topk)
    csv_path, json_path = save_outputs(df, results)
    msg = f"Processed {len(results)} files. Showing top {min(topk, len(results))}."
    return msg, df, csv_path, json_path

with gr.Blocks(title="CV/LinkedIn Matcher") as demo:
    gr.Markdown("## 🔎 CV / LinkedIn Matcher\nUpload CVs and/or LinkedIn screenshots. JD is optional.")
    with gr.Row():
        files = gr.File(label="Upload files (.pdf, .docx, .txt, .rtf, .png, .jpg, .jpeg, .webp)", file_count="multiple")
        jd = gr.Textbox(label="Job Description (optional)", value=DEFAULT_JD, lines=6, placeholder="Leave blank to skip JD-based matching.")
        topk = gr.Slider(1, 50, value=10, step=1, label="Top N to show / export")
    run_btn = gr.Button("Run matching", variant="primary")
    status = gr.Markdown()
    table = gr.Dataframe(interactive=False, wrap=True, label="Top Matches Preview")
    csv_file = gr.File(label="Download CSV")
    json_file = gr.File(label="Download JSON (top N)")

    run_btn.click(fn=run_match, inputs=[files, jd, topk], outputs=[status, table, csv_file, json_file])

demo.launch(debug=True)  # in Kaggle this shows inline; no need for share=True


2025-08-24 10:33:04.322037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756031584.345527      94 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756031584.352703      94 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://54e9ecfa6622e51244.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading Qwen 2.5 1.5B model...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

All models loaded successfully!
